# IMPORTACIONES

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import joblib

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# LECTURA DE DATOS

In [ ]:
df_students = pd.read_csv('Kaggle_Dataset.csv')
df_students.head()

# 1 ANÁLISIS EXPLORATORIO DE DATOS (EDA)

In [ ]:
df_students.info()

In [ ]:
display(df_students.describe().T)

In [ ]:
df_students.nunique()

## 1.1 TRANSFORMACIÓN A CATEGÓRICAS
Las variables categóricas vienen codificadas en numéricas. El primer paso que vamos a realizar es crear diccionarios para dar nombre a las categorias y, de esta forma, poder realizar un primer análisis preliminar que nos ayude a comprender mejor las variables y el dataset.

In [ ]:
# Marital status
marital_status_map = {
    1: "single",
    2: "married",
    3: "widow",
    4: "divorced",
    5: "facto union",
    6: "legally separated"
}

# Application mode
application_mode_map = {
    1: "1st phase - general contingent",
    2: "Ordinance No. 612/93",
    3: "1st phase - special contingent (Azores Island)",
    4: "Holders of other higher courses",
    5: "Ordinance No. 854-B/99",
    6: "International student (bachelor)",
    7: "1st phase - special contingent (Madeira Island)",
    8: "2nd phase - general contingent",
    9: "3rd phase - general contingent",
    10: "Ordinance No. 533-A/99, item b2) (Different Plan)",
    11: "Ordinance No. 533-A/99, item b3 (Other Institution)",
    12: "Over 23 years old",
    13: "Transfer",
    14: "Change of course",
    15: "Technological specialization diploma holders",
    16: "Change of institution/course",
    17: "Short cycle diploma holders",
    18: "Change of institution/course (International)"
}

# Application order
application_order_map = {i: f"choice {i}" for i in range(0, 22)}

# Course
course_map = {
    1: "Biofuel Production Technologies",
    2: "Animation and Multimedia Design",
    3: "Social Service (evening attendance)",
    4: "Agronomy",
    5: "Communication Design",
    6: "Veterinary Nursing",
    7: "Informatics Engineering",
    8: "Equiniculture",
    9: "Management",
    10: "Social Service",
    11: "Tourism",
    12: "Nursing",
    13: "Oral Hygiene",
    14: "Advertising and Marketing Management",
    15: "Journalism and Communication",
    16: "Basic Education",
    17: "Management (evening attendance)"
}


# Daytime/evening attendance
daytime_map = {
    1: "daytime",
    0: "evening"
}

# Previous qualification
previous_qualification_map = {
    1: "Secondary education",
    2: "Higher education - bachelor's degree",
    3: "Higher education - degree",
    4: "Higher education - master's",
    5: "Higher education - doctorate",
    6: "Frequency of higher education",
    7: "12th year of schooling - not completed",
    8: "11th year of schooling - not completed",
    9: "Other - 11th year of schooling",
    10: "10th year of schooling",
    11: "10th year of schooling - not completed",
    12: "Basic education 3rd cycle (9th/10th/11th year) or equiv.",
    13: "Basic education 2nd cycle (6th/7th/8th year) or equiv.",
    14: "Technological specialization course",
    15: "Higher education - degree (1st cycle)",
    16: "Professional higher technical course",
    17: "Higher education - master (2nd cycle)"
}


# Nacionality
nacionality_map = {
    1: 'Portuguese',
    2: 'German',
    3: 'Spanish',
    4: 'Italian',
    5: 'Dutch',
    6: 'English',
    7: 'Lithuanian',
    8: 'Angolan',
    9: 'Cape Verdean',
    10: 'Guinean',
    11: 'Mozambican',
    12: 'Santomean',
    13: 'Turkish',
    14: 'Brazilian',
    15: 'Romanian',
    16: 'Moldova',
    17: 'Mexican',
    18: 'Ukrainian',
    19: 'Russian',
    20: 'Cuban',
    21: 'Colombian'
}

# Qualifications parents
mother_qualification_map = {
    1: "Secondary Education - 12th Year of Schooling or Eq.",
    2: "Higher Education - Bachelor's Degree",
    3: "Higher Education - Degree",
    4: "Higher Education - Master's",
    5: "Higher Education - Doctorate",
    6: "Frequency of Higher Education",
    7: "12th Year of Schooling - Not Completed",
    8: "11th Year of Schooling - Not Completed",
    9: "7th Year (Old)",
    10: "Other - 11th Year of Schooling",
    11: "10th Year of Schooling",
    12: "General commerce course",
    13: "Basic Education 3rd Cycle (9th/10th/11th Year) or Equiv.",
    14: "Technical-professional course",
    15: "7th year of schooling",
    16: "2nd cycle of the general high school course",
    17: "9th Year of Schooling - Not Completed",
    18: "8th year of schooling",
    19: "Unknown",
    20: "Can't read or write",
    21: "Can read without having a 4th year of schooling",
    22: "Basic education 1st cycle (4th/5th year) or equiv.",
    23: "Basic Education 2nd Cycle (6th/7th/8th Year) or Equiv.",
    24: "Technological specialization course",
    25: "Higher education - degree (1st cycle)",
    26: "Specialized higher studies course",
    27: "Professional higher technical course",
    28: "Higher Education - Master (2nd cycle)",
    29: "Higher Education - Doctorate (3rd cycle)"
}

father_qualification_map = {
    1: "Secondary Education - 12th Year of Schooling or Equiv.",
    2: "Higher Education - Bachelor's Degree",
    3: "Higher Education - Degree",
    4: "Higher Education - Master's",
    5: "Higher Education - Doctorate",
    6: "Frequency of Higher Education",
    7: "12th Year of Schooling - Not Completed",
    8: "11th Year of Schooling - Not Completed",
    9: "7th Year (Old)",
    10: "Other - 11th Year of Schooling",
    11: "2nd year complementary high school course",
    12: "10th Year of Schooling",
    13: "General commerce course",
    14: "Basic Education 3rd Cycle (9th/10th/11th Year) or Equiv.",
    15: "Complementary High School Course",
    16: "Technical-professional course",
    17: "Complementary High School Course - not concluded",
    18: "7th year of schooling",
    19: "2nd cycle of the general high school course",
    20: "9th Year of Schooling - Not Completed",
    21: "8th year of schooling",
    22: "General Course of Administration and Commerce",
    23: "Supplementary Accounting and Administration",
    24: "Unknown",
    25: "Can't read or write",
    26: "Can read without having a 4th year of schooling",
    27: "Basic education 1st cycle (4th/5th year) or Equiv.",
    28: "Basic Education 2nd Cycle (6th/7th/8th Year) or Equiv.",
    29: "Technological specialization course",
    30: "Higher education - degree (1st cycle)",
    31: "Specialized higher studies course",
    32: "Professional higher technical course",
    33: "Higher Education - Master (2nd cycle)",
    34: "Higher Education - Doctorate (3rd cycle)"
}


# Occupations parents 
mother_occupation_map = {
    1: "Student",
    2: "Representatives of the Legislative Power and Executive Bodies, Directors, Directors and Executive Managers",
    3: "Specialists in Intellectual and Scientific Activities",
    4: "Intermediate Level Technicians and Professions",
    5: "Administrative staff",
    6: "Personal Services, Security and Safety Workers and Sellers",
    7: "Farmers and Skilled Workers in Agriculture, Fisheries and Forestry",
    8: "Skilled Workers in Industry, Construction and Craftsmen",
    9: "Installation and Machine Operators and Assembly Workers",
    10: "Unskilled Workers",
    11: "Armed Forces Professions",
    12: "Other Situation",
    13: "(blank)",
    14: "Health professionals",
    15: "teachers",
    16: "Specialists in information and communication technologies (ICT)",
    17: "Intermediate level science and engineering technicians and professions",
    18: "Technicians and professionals, of intermediate level of health",
    19: "Intermediate level technicians from legal, social, sports, cultural and similar services",
    20: "Office workers, secretaries in general and data processing operators",
    21: "Data, accounting, statistical, financial services and registry-related operators",
    22: "Other administrative support staff",
    23: "personal service workers",
    24: "sellers",
    25: "Personal care workers and the like",
    26: "Skilled construction workers and the like, except electricians",
    27: "Skilled workers in printing, precision instrument manufacturing, jewelers, artisans and the like",
    28: "Workers in food processing, woodworking, clothing and other industries and crafts",
    29: "cleaning workers",
    30: "Unskilled workers in agriculture, animal production, fisheries and forestry",
    31: "Unskilled workers in extractive industry, construction, manufacturing and transport",
    32: "Meal preparation assistants"
}

father_occupation_map = {
    0: "Student",
    1: "Representatives of the Legislative Power and Executive Bodies, Directors, Directors and Executive Managers",
    2: "Specialists in Intellectual and Scientific Activities",
    3: "Intermediate Level Technicians and Professions",
    4: "Administrative staff",
    5: "Personal Services, Security and Safety Workers and Sellers",
    6: "Farmers and Skilled Workers in Agriculture, Fisheries and Forestry",
    7: "Skilled Workers in Industry, Construction and Craftsmen",
    8: "Installation and Machine Operators and Assembly Workers",
    9: "Unskilled Workers",
    10: "Armed Forces Professions",
    11: "Armed Forces Officers",
    12: "Armed Forces Sergeants",
    13: "Other Armed Forces personnel",
    14: "Directors of administrative and commercial services",
    15: "Hotel, catering, trade and other services directors",
    16: "Specialists in the physical sciences, mathematics, engineering and related techniques",
    17: "Health professionals",
    18: "Teachers",
    19: "Specialists in finance, accounting, administrative organization, public and commercial relations",
    20: "Intermediate level science and engineering technicians and professions",
    21: "Technicians and professionals, of intermediate level of health",
    22: "Intermediate level technicians from legal, social, sports, cultural and similar services",
    23: "Information and communication technology technicians",
    24: "Office workers, secretaries in general and data processing operators",
    25: "Data, accounting, statistical, financial services and registry-related operators",
    26: "Other administrative support staff",
    27: "Personal service workers",
    28: "Sellers",
    29: "Personal care workers and the like",
    30: "Protection and security services personnel",
    31: "Market-oriented farmers and skilled agricultural and animal production workers",
    32: "Farmers, livestock keepers, fishermen, hunters and gatherers, subsistence",
    33: "Skilled construction workers and the like, except electricians",
    34: "Skilled workers in metallurgy, metalworking and similar",
    35: "Skilled workers in electricity and electronics",
    36: "Workers in food processing, woodworking, clothing and other industries and crafts",
    37: "Fixed plant and machine operators",
    38: "Assembly workers",
    39: "Vehicle drivers and mobile equipment operators",
    40: "Unskilled workers in agriculture, animal production, fisheries and forestry",
    41: "Unskilled workers in extractive industry, construction, manufacturing and transport",
    42: "Meal preparation assistants",
    43: "Street vendors (except food) and street service providers"
}


# Displaced
displaced_map = {
    1: "yes",
    0: "no"
}

# Educational special needs
special_needs_map = {
    1: "yes",
    0: "no"
}

# Debtor
debtor_map = {
    1: "yes",
    0: "no"
}

# Tuition fees up to date
tuition_map = {
    1: "yes",
    0: "no"
}

# Gender
gender_map = {
    1: "male",
    0: "female"
}

# Scholarship holder
scholarship_map = {
    1: "yes",
    0: "no"
}

# International
international_map = {
    1: "yes",
    0: "no"
}

# Target
target_map = {
    "Dropout": "dropout",
    "Enrolled": "still enrolled",
    "Graduate": "graduate"
}


In [ ]:
mappings = {
    "Marital status": marital_status_map,
    "Application mode": application_mode_map,
    "Application order": application_order_map,
    "Course": course_map,
    "Daytime/evening attendance": daytime_map,
    "Previous qualification": previous_qualification_map,
    "Nacionality": nacionality_map,
    "Mother's qualification": mother_qualification_map,
    "Father's qualification": father_qualification_map,
    "Mother's occupation": mother_occupation_map,
    "Father's occupation": father_occupation_map,
    "Displaced": displaced_map,
    "Educational special needs": special_needs_map,
    "Debtor": debtor_map,
    "Tuition fees up to date": tuition_map,
    "Gender": gender_map,
    "Scholarship holder": scholarship_map,
    "International": international_map,
    "Target": target_map
}

for col, mapping in mappings.items():
    if col in df_students.columns:
        df_students[col] = df_students[col].map(mapping).astype("category")


In [ ]:
mas_comun = df_students["Father's occupation"].mode()[0]

df_students["Father's occupation"].fillna(mas_comun, inplace=True)


In [ ]:
df_students.info()

## 1.2 ANÁLISIS DE VARIABLES

### 1.2.1 ANÁLISIS TARGET
El target es multiclase. Vamos a realizar 1 tipo de predicción, uno binario eliminando aquellos que siguen cursando el curso, ya que estos no aportan ningún tipo de valor real actual, ni en el interes de predicirlo ni en datos útiles.

In [ ]:
print("\n📊 Conteo de valores de 'target':\n")
print(df_students["Target"].value_counts())

In [ ]:
fig = px.histogram(df_students, x="Target") 
fig.show()

Vamos a eliminar la información procedente de "still enrolled" porque no tenemos información de si esa población acabará abandonando o no. Nos podemos proponer al final del proyecto tratar de ver si podemos predecir si continuan estudiando,

In [ ]:
df_students = df_students[df_students.Target != 'still enrolled']
df_students['Target'] = df_students['Target'].cat.remove_categories('still enrolled')

In [ ]:
df_students.shape

In [ ]:
column_name = 'Target'

df_students = df_students[df_students[column_name] != 'Enrolled']

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
sns.countplot(y=column_name, data=df_students, palette='Set3')  
plt.title(f'Distribución de la {column_name}')

ax = plt.gca()
for p in ax.patches:
    ax.annotate(
        f'{int(p.get_width())}', 
        (p.get_width(), p.get_y() + p.get_height() / 2), 
        ha='center', 
        va='center', 
        xytext=(10, 0), 
        textcoords='offset points'
    )

sns.despine(left=True, bottom=True)

plt.subplot(1, 2, 2)
counts = df_students[column_name].value_counts()
explode = [0.05] * len(counts)  

counts.plot.pie(
    autopct='%1.1f%%', 
    colors=sns.color_palette('Set3'), 
    startangle=90, 
    explode=explode
)
plt.title(f'% de la distribución {column_name}')
plt.ylabel('') 

plt.tight_layout()
plt.show()

Al observar la variable objetivo, se identifican 2.209 estudiantes graduados frente a 1.421 estudiantes en situación de abandono.

- Esto implica que el dataset está moderadamente desbalanceado (aprox. 61% graduados vs 39% abandonos).
- No es un desbalance extremo, pero sí relevante: un modelo trivial que predijera siempre “graduate” ya alcanzaría ~61% de accuracy.
- Por lo tanto, métricas adicionales como recall y precisión en la clase dropout son críticas, ya que el foco principal es identificar a los estudiantes en riesgo de abandono.
- Además, al no estar completamente equilibrado, conviene monitorizar el recall de dropout para evitar que el modelo priorice solo a la clase mayoritaria (graduados).

### 1.2.2 ANÁLISIS VARIABLES CATEGORICAS CON > 10 VALORES
Primero vemos aquellas categorias que tengan mas de 10 valores únicos 

In [ ]:

categorical_cols = df_students.select_dtypes(['category']).columns

def summarize_category(df, col, top=50):
    vc = df[col].value_counts(normalize=True)
    top_categories = vc.head(top)
    others = vc[top:].sum()
    summary = pd.concat([top_categories, pd.Series({'Other': others})])
    return summary

def show_top_categories(df, col, top=50):
    summary = summarize_category(df, col, top)
    print(f"\nVariable: {col} (total {df[col].nunique()} categorías)")
    display(pd.DataFrame(summary*100, columns=['% de registros']))

def plot_top_categories(df, col, top=50):
    summary = summarize_category(df, col, top)
    sns.barplot(x=summary.values, y=summary.index, palette="viridis")
    plt.xlabel("Proporción")
    plt.title(f"Distribución de '{col}' (top {top} + Other)")
    plt.show()

for col in categorical_cols:
    if df_students[col].nunique() > 10:
        show_top_categories(df_students, col, top=50)
        plot_top_categories(df_students, col, top=50)


#### Procesamiento en las categorias con mas de 10 variables

##### Algunas de estas categorias tienen una distribución bastante desigual, con algunos valores minimamente representados entorpeciendo el analisis y modelaje. Para ello antes de describir insights en el EDA, vamos a hacer un pequeño preprocesamiento para obtener un análisis mas ajustado. La idea es tratar de tener todas las variable categoricas con 10 o menos valores distintos en los casos en los que sea eficiente y razonable hacerlo.

1. Application mode
- Tiene 18 variables pero el 80% de los casos están agrupados en 4 valores. Vamos reducir la cardinalidad de esta variable agrupando todos los valores por debajo del 1% de representatividad en un grupo denominado "Others". Probaremos los resultados de esta forma y planteamos una posible opcion b en el que los agruparemos en funcion de su tipo de aplicación.

2. Course
- En el caso de la variable Course, aunque presenta 17 categorías distintas, su distribución se encuentra relativamente balanceada: la mayoría de los programas académicos concentran entre un 3% y un 8% de los registros, con solo dos titulaciones minoritarias por debajo del 2%.

Dado que cada categoría representa una carrera universitaria específica —con un significado propio y difícilmente agrupable con otras— no resulta adecuado reducir su cardinalidad de manera artificial. Eliminar o fusionar titulaciones podría llevar a la pérdida de información relevante, ya que cada programa académico puede presentar patrones particulares de rendimiento o abandono.

Por ello, he decidido mantener la variable con todas sus categorías originales.

3. Previous qualification
- En este caso vemos una variable claramente desbalanceada por una única opción: "Secondary education", que abarca el 84% de la distribución. En este caso para tratar de no perder nada de información de momento vamos a dividirlas en 4 bloques:
    - Secondary education (84%)
    - Basic education (sumando 9º–11º, 2º ciclo, incompletos, etc.)
    - Higher education (grado, máster, doctorado, frequency of HE…)
    - Other / Specialization (tecnológicos, técnicos, cursos profesionales).
- En caso de que no aporten suficiente información probaremos a agrupar el resto en others obteniendo tan solo 2 valores.

4. Nationality
- La variable Nacionality está fuertemente desbalanceada, con el 97,5% de los registros correspondientes a “Portuguese” y el resto de nacionalidades representando únicamente un 2,5% del total, muchas con menos del 0,1% de los casos.
Dada su baja variabilidad y su limitada aportación al análisis y modelado, se ha decidido eliminar esta variable del dataset. Esta decisión simplifica el preprocesamiento sin comprometer la información relevante.

5. Mother's qualification
- La variable Mother's qualification presenta 29 categorías, muchas de ellas con representatividad extremadamente baja (<0,1%), mientras que las categorías principales concentran la gran mayoría de los registros. Para reducir la cardinalidad y mantener la interpretabilidad de la variable, se ha decidido agrupar las categorías en bloques educativos lógicos: Basic Education, Secondary Education, Higher Education y Unknown/Other.

Esta decisión permite preservar la información relevante sobre el nivel educativo de la madre —que puede influir en el desempeño del estudiante— evitando el ruido generado por categorías residuales con escasa representación. Además, facilita la interpretación de los resultados tanto en el análisis exploratorio como en las fases de modelado.

6. Father's qualification
- La variable Father's qualification presenta 34 categorías, muchas de ellas con representatividad extremadamente baja (<0,1%), mientras que unas pocas categorías concentran la mayoría de los registros (~85–90%). Para reducir la cardinalidad y mantener la interpretabilidad de la variable, se ha decidido agrupar las categorías en bloques educativos lógicos: Basic Education, Secondary Education, Higher Education y Unknown/Other.

Esta decisión permite preservar la información relevante sobre el nivel educativo del padre —que puede influir en el desempeño del estudiante— evitando el ruido generado por categorías residuales con escasa representación. Además, facilita la interpretación de los resultados tanto en el análisis exploratorio como en las fases de modelado.

7. Mother's occupation
- La variable Mother's occupation presenta 32 categorías, con una distribución altamente desigual: unas pocas categorías concentran la mayoría de los registros (≈80%), mientras que muchas otras presentan representatividad muy baja (<1%).

Para reducir la cardinalidad y mantener la interpretabilidad de la variable, se ha decidido agrupar las ocupaciones en bloques amplios y coherentes, según el tipo de actividad y nivel de especialización:

8. Father's occupation
- La variable Father's occupation presenta 43 categorías, con una distribución muy desigual: unas pocas categorías concentran la mayoría de los registros (≈80%), mientras que muchas otras tienen representación extremadamente baja (<1%).

Para reducir la cardinalidad y mantener la interpretabilidad de la variable, se ha decidido agrupar las ocupaciones en bloques amplios y coherentes, según el tipo de actividad y nivel de especialización:

#### 1.2.2.1 PREPROCESADO DE LAS VARIABLES CATEGÓRICAS CON >10 VALORES

In [ ]:
# 1. Application mode: unificamos categorías con muy poca representación (<1%) en una sola categoría "Others"
app_mode_counts = df_students['Application mode'].value_counts(normalize=True)
small_categories = app_mode_counts[app_mode_counts < 0.01].index
df_students['Application mode'] = df_students['Application mode'].replace(small_categories, 'Others')


# 2. Previous qualification: agrupamos los niveles de estudios previos en grandes bloques (Basic, Secondary, Higher).
prev_edu_map = {
    'Secondary education': 'Secondary',
    'Basic education 3rd cycle (9th/10th/11th year) or equiv.': 'Basic',
    'Basic education 2nd cycle (6th/7th/8th year) or equiv.': 'Basic',
    'Basic education 1st cycle (...)': 'Basic', 
    'Higher education - degree': 'Higher',
    'Higher education - bachelor\'s degree': 'Higher',
    'Higher education - master (2nd cycle)': 'Higher',
}
df_students['Previous qualification'] = df_students['Previous qualification'].map(prev_edu_map).fillna('Other/Spec')


# 3. Mother's qualification: agrupamos en tres grandes niveles (Basic, Secondary, Higher).
mother_qual_map = {
    'Basic education 1st cycle (...)': 'Basic',
    'Basic education 2nd cycle (...)': 'Basic',
    'Basic education 3rd cycle (...)': 'Basic',
    'Secondary Education - 12th Year of Schooling or Eq.': 'Secondary',
    'Higher Education - Degree': 'Higher',
    'Higher Education - Bachelor\'s Degree': 'Higher',
    'Unknown': 'Other/Unknown'
}
df_students["Mother's qualification"] = df_students["Mother's qualification"].map(mother_qual_map).fillna('Other/Unknown')


# 4. Father's qualification: mismo criterio que para la madre, agrupando en Basic, Secondary y Higher.
father_edu_map = {
    # Basic
    'Basic education 1st cycle (4th/5th year) or Equiv.': 'Basic',
    'Basic Education 3rd Cycle (9th/10th/11th Year) or Equiv.': 'Basic',
    'Basic Education 2nd Cycle (6th/7th/8th Year) or Equiv.': 'Basic',
    '7th Year (Old)': 'Basic',
    '10th Year of Schooling': 'Basic',
    '11th Year of Schooling - Not Completed': 'Basic',
    '9th Year of Schooling - Not Completed': 'Basic',
    'Can read without having a 4th year of schooling': 'Basic',
    '8th year of schooling': 'Basic',
    '7th year of schooling': 'Basic',
    '10th year of schooling - not completed': 'Basic',
    '12th Year of Schooling - Not Completed': 'Basic',
    
    # Secondary
    'Secondary Education - 12th Year of Schooling or Equiv.': 'Secondary',
    'Other - 11th Year of Schooling': 'Secondary',

    # Higher
    'Higher Education - Degree': 'Higher',
    "Higher Education - Bachelor's Degree": 'Higher',
    "Higher Education - Master's": 'Higher',
    'Higher education - degree (1st cycle)': 'Higher',
    'Higher Education - Master (2nd cycle)': 'Higher',
    'Higher Education - Doctorate': 'Higher',
    'Frequency of Higher Education': 'Higher',
    
    # Unknown / Other
    'Unknown': 'Unknown/Other',
    'Professional higher technical course': 'Unknown/Other',
    'Technological specialization course': 'Unknown/Other'
}
df_students["Father's qualification"] = df_students["Father's qualification"].map(father_edu_map).fillna('Unknown/Other')


# 5. Mother's occupation: reducimos a grandes grupos (Unskilled/Manual, Admin/Services, Specialized/Prof).
mother_occ_map = {
    'Unskilled Workers': 'Unskilled/Manual',
    'Administrative staff': 'Admin/Services',
    'Personal Services, Security and Safety Workers and Sellers': 'Admin/Services',
    'Intermediate Level Technicians and Professions': 'Specialized/Prof',
    'Specialists in Intellectual and Scientific Activities': 'Specialized/Prof',
}
df_students["Mother's occupation"] = df_students["Mother's occupation"].map(mother_occ_map).fillna('Other/Unknown')


# 6. Father's occupation: igual estrategia que para la madre, simplificando en bloques generales.
father_occ_map = {
    'Armed Forces Professions': 'Armed Forces',
    'Installation and Machine Operators and Assembly Workers': 'Unskilled/Manual',
    'Farmers and Skilled Workers in Agriculture, Fisheries and Forestry': 'Unskilled/Manual',
    'Personal Services, Security and Safety Workers and Sellers': 'Admin/Services',
    'Administrative staff': 'Admin/Services',
    'Unskilled Workers': 'Unskilled/Manual',
    'Armed Forces Officers': 'Armed Forces',
    'Skilled Workers in Industry, Construction and Craftsmen': 'Unskilled/Manual',
    'Intermediate Level Technicians and Professions': 'Specialized/Prof',
}
df_students["Father's occupation"] = df_students["Father's occupation"].map(father_occ_map).fillna('Other/Unknown')

### 1.2.3 ANÁLISIS DE TODAS LAS VARIABLES CATEGÓRICAS

In [ ]:
def plot_distribucion_categoricas(column_name, data):
    plt.figure(figsize=(12,5))

    plt.subplot(1, 2, 1)
    ax = sns.countplot(
        x=column_name,
        data=data,
        order=data[column_name].value_counts().index,
        palette="viridis"
    )
    plt.title(f'Distribution of {column_name}')
    plt.xticks(rotation=45)
    sns.despine(left=True, bottom=True)

    for p in ax.patches:
        ax.annotate(
            f'{p.get_height()}',
            (p.get_x() + p.get_width() / 2., p.get_height()),
            ha='center', va='bottom',
            fontsize=10, color='black', xytext=(0, 5),
            textcoords='offset points'
        )

    plt.subplot(1, 2, 2)
    counts = data[column_name].value_counts()
    counts.plot.pie(
        autopct='%1.1f%%',
        colors=sns.color_palette('Set3'),
        startangle=90,
        explode=[0.05]*len(counts)
    )
    plt.title(f'Percentage Distribution of {column_name}')
    plt.ylabel('')
    plt.show()

categoricas = df_students.select_dtypes(include=['object', 'category']).columns.tolist()

for feature in categoricas:
    plot_distribucion_categoricas(feature, df_students)

#### 1.2.3.1 INSIGHTS VARIABLES CATEGÓRICAS
1. "Marital Status":
- La mayoría de los estudiantes se encuentra soltera/o (≈72%), seguida por los casados (≈7%). Las demás categorías —divorciado, unión de hecho, legalmente separado y viudo— presentan una representación muy baja (<2%).
- La variable está altamente desbalanceada, por lo que para análisis o modelado futuro podría considerarse agrupar las categorías minoritarias en un bloque “Otros” para simplificar y evitar ruido, aunque la categoría mayoritaria probablemente capture la mayoría de patrones relevantes.

2. "Application mode":
- La mayor parte de los estudiantes se concentra en las primeras fases de admisión: 1st phase - general contingent (≈32%) y 2nd phase - general contingent (≈16%). Otros grupos significativos incluyen Over 23 years old (≈15%) y Change of course (≈5%). Las restantes categorías representan menos del 5% de los registros cada una, siendo Transfer y Change of institution/course las menos frecuentes (<2%).
- La variable está parcialmente desbalanceada. Para análisis o modelado futuro, podría ser útil agrupar las categorías con baja representación en un bloque “Others”, mientras se mantiene el detalle de los grupos más grandes para capturar patrones diferenciados de aplicación.

3. "Application order":
- La gran mayoría de los estudiantes ha seleccionado su primera opción como prioridad: choice 1 representa ≈55% de los registros. Las opciones posteriores disminuyen progresivamente: choice 2 (≈10%), choice 3 (≈6%), choice 4 (≈5%), hasta llegar a las menos frecuentes choice 6 y choice 0, con menos del 3% combinadas. Choice 9 no tiene registros.
- La variable refleja una fuerte preferencia por la primera opción de aplicación, con una caída pronunciada en las opciones subsecuentes.

4. "Course"
- Nursing domina con ≈18%, seguido por Social Service, Journalism y Management (≈7–9%).
- Carreras tecnológicas como Informatics Engineering o Biofuel Production Technologies son minoritarias (<3%).
- Esto sugiere que la institución se orienta principalmente hacia salud y ciencias sociales, con baja atracción en ingenierías.

5. "Daytime/evening attendance":
- La gran mayoría cursa en horario diurno (≈89%), mientras que solo un 11% asiste en la modalidad de tarde/noche.
- La variable está fuertemente desbalanceada y refleja que el perfil predominante son estudiantes de dedicación completa; la categoría “evening” podría tener importancia en casos de estudiantes trabajadores.

6. "Previous qualification":
- Predomina la formación secundaria (≈82%), con menor representación de “Other/Spec” (≈8%), básica (≈4%) y superior (≈4%).
- El fuerte peso de la secundaria indica que la mayoría accede directamente desde este nivel. Las categorías minoritarias podrían reagruparse en “Otros” en caso de simplificación.

7. "Nationality":
- La población estudiantil es mayoritariamente portuguesa (≈98%), con muy poca presencia internacional (menos del 2% repartido en múltiples países).
- La variable está extremadamente desbalanceada, lo que la hace poco útil para el modelado predictivo; podría binarizarse como “portugués / no portugués”.

8. "Mother's qualification":
- Predomina la categoría Other/Unknown (≈60%), seguida de secundaria (≈22%) y superior (≈10%).
- El elevado peso de datos faltantes (“Other/Unknown”) limita el valor informativo; puede requerir imputación o ser descartada si no aporta al modelo.

9. "Father's qualification":
- La mayoría de los padres posee educación básica (≈66%), con secundaria (≈21%) y superior (≈9%).
- La distribución sugiere un perfil parental de bajo nivel educativo, lo que podría estar relacionado con la tasa de éxito o permanencia de los estudiantes.

10. "Mother's occupation":
- Las categorías más frecuentes son Unskilled/Manual (≈37%) y Admin/Services (≈31%), seguidas por Other/Unknown (≈20%) y Specialized/Prof (≈14%).
- La variable está algo desbalanceada, pero conserva suficiente información sobre el nivel ocupacional y profesional de la madre. Puede ser útil para analizar correlaciones con el desempeño estudiantil, aunque las categorías minoritarias podrían reagruparse si se busca simplificar el modelo.

11. "Father's occupation":
- La mayoría de los registros corresponde a Unskilled/Manual (≈41%) y Armed Forces (≈31%), con menor representación de Admin/Services (≈18%), Other/Unknown (≈10%) y Specialized/Prof (≈4%).
- La variable refleja un perfil profesional predominante en trabajos manuales y fuerzas armadas. La baja representación de las categorías minoritarias podría permitir agruparlas en un bloque “Otros” para simplificación sin perder información relevante.

12. "Displaced":
- Aproximadamente el 55% de los estudiantes están desplazados (viven fuera de su localidad de origen). Esto puede afectar su integración social y desempeño académico, por lo que podría ser relevante para predecir graduación o abandono.

13. "Educational special needs":
- Solo el 1% de los estudiantes presenta necesidades educativas especiales, indicando que la gran mayoría sigue la educación estándar. Esto limita la variabilidad de la variable, aunque los casos identificados podrían requerir apoyo adicional.

14. "Debtor":
- Alrededor del 11% de los estudiantes tiene deudas pendientes. Esto puede reflejar dificultades financieras que incrementen el riesgo de abandono.

15. "Tuition fees up to date"
- El 87% de los estudiantes tiene las cuotas al día. Mantener los pagos completos puede ser un indicador de estabilidad económica y compromiso con la continuidad académica.

16. "Gender"
- La distribución es aproximadamente 66% mujeres y 34% hombres. Diferencias de género pueden estar asociadas a patrones de rendimiento o permanencia en el programa, aunque no necesariamente determinantes.

17. "Scholarship holder"
- Cerca del 27% de los estudiantes recibe beca. Esto podría relacionarse con el rendimiento académico o la permanencia, reflejando apoyo financiero que facilita la continuidad del estudiante.

18. "International"
- Solo el 2% de los estudiantes son internacionales. Esto indica que la universidad está mayoritariamente compuesta por estudiantes locales, aunque los internacionales podrían enfrentar retos adicionales de adaptación y permanencia.

19. "Target"
- Distribución ≈61% graduados y 39% abandonos. Esta variable es el objetivo principal y permite evaluar la relación de todas las anteriores con la permanencia o deserción académica.

#### 1.2.3.2 DROP
En este momento hacemos una primera selección de variables inútiles para el modelo en función de estos primeros insights
Variables descartadas:
1. "Nationality":
- La gran mayoría de los estudiantes (≈97,5%) son portugueses, mientras que el resto de nacionalidades tiene una representación muy baja (<3%), muchas con uno o ningún registro.
- Debido a esta alta desproporción, la variable aporta muy poca información discriminativa para predecir si un estudiante se gradúa o abandona.
- Mantenerla podría introducir ruido o generar codificaciones innecesarias, por lo que se descarta del análisis y del modelado.

In [ ]:
df_students.drop("Nacionality",axis=1,inplace=True)

2. "Educational special needs":
- Tiene 40 registros “yes” vs 3590 “no”.
- Si bien refleja una característica importante, su extremadamente baja representación podría hacer que el modelo no aprenda patrones relevantes. Se podría mantener si interesa estudiar casos especiales, o eliminar si se quiere simplificar.

In [ ]:
df_students.drop("Educational special needs",axis=1,inplace=True)

3. "International"
- Solo 86 estudiantes son internacionales. Podría ser irrelevante si la variable no aporta mucho al modelo y se quiere simplificar.

In [ ]:
df_students.drop("International",axis=1,inplace=True)

### 1.2.4 VARIABLES NUMÉRICAS

In [ ]:
def plot_distribucion_mumerica(column_name, data):
    plt.figure(figsize=(10, 4))

    plt.subplot(1, 2, 1)
    sns.histplot(data[column_name], bins=30, kde=True, color='skyblue')
    plt.title(f'Distribution of {column_name}')

    plt.subplot(1, 2, 2)
    sns.boxplot(x=data[column_name], color='lightgreen')
    plt.title(f'Boxplot of {column_name}')

    plt.tight_layout()
    plt.show()


numeric_features = df_students.select_dtypes(include=['int64','float64']).columns.tolist()

for feature in numeric_features:
    plot_distribucion_mumerica(feature, df_students)


In [ ]:
numerical_df = df_students.select_dtypes(include=[np.number])

summary = pd.DataFrame({
    "mean": numerical_df.mean(),
    "median": numerical_df.median(),
    "std": numerical_df.std(),
    "min": numerical_df.min(),
    "max": numerical_df.max(),
    "skewness": numerical_df.skew(),
    "kurtosis": numerical_df.kurt(),
    "nulos": numerical_df.isnull().sum()
})

summary = summary.round(3)

display(summary)


#### 1.2.4.1 INSIGHTS VARIABLES NUMÉRICAS
1. Age at enrollment:
- La distribución está positivamente sesgada (skewness = 1.991), lo que indica que aunque la mayoría de los estudiantes ingresan en edades tempranas (cerca de 20 años), existe un grupo de estudiantes significativamente mayor. En el histograma observamos claramente como 
- Insight: Los estudiantes más mayores podrían enfrentar responsabilidades adicionales, como trabajo o familia, que podrían influir en su rendimiento académico o propensión a abandonar. Por el contrario, los más jóvenes podrían adaptarse más fácilmente al entorno universitario.
2. Curricular units 1st sem (credited)
- Sesgo positivo elevado (skewness ≈ 4.06) y kurtosis muy alta (≈ 17.99), lo que indica que la mayoría de los estudiantes tienen 0 créditos, pero unos pocos logran muchos créditos.
- Insight: Los estudiantes que logran acumular créditos tempranamente muestran alto compromiso y probablemente menor riesgo de abandono (lo comprobaremos mas adelante), mientras que los que no logran créditos iniciales podrían necesitar apoyo académico.
- Tiene una distribución extremadamente sesgada y curtosis altísima. La mayoría de estudiantes tiene 0, con unos pocos valores extremos. Esto aporta muy poca información general para el modelo, y puede actuar más como ruido que como predictor confiable. Podrías agruparlo, transformarlo o incluso descartarlo.
3. Curricular units 1st sem (enrolled)
- Sesgo positivo (≈ 1.65) sugiere que algunos estudiantes se inscriben en una carga excesiva de unidades.
- Insight: Una carga muy alta puede generar estrés y mayor riesgo de fracaso o abandono; la mayoría mantiene una carga promedio, lo que refleja planificación académica estándar.
4. Curricular units 1st sem (evaluations)
- Sesgo positivo moderado (≈ 1.15), con estudiantes enfrentando muchas evaluaciones.
- Insight: Un número elevado de evaluaciones podría indicar cursos más exigentes. Aquellos con muchas evaluaciones podrían tener mayor presión académica, lo que puede correlacionarse con riesgo de abandono si no gestionan bien su tiempo.
5. Curricular units 1st sem (approved)
- Sesgo ligeramente positivo (≈ 0.75), con la mayoría aprobando alrededor de 5 unidades.
- Insight: El desempeño en el primer semestre es un indicador temprano del éxito académico. Los estudiantes con pocas aprobaciones iniciales podrían estar en riesgo de abandono.
6. Curricular units 1st sem (grade)
- Sesgo negativo (≈ -1.45), lo que indica que la mayoría tiene notas altas, pero algunos estudiantes tienen calificaciones muy bajas.
- Insight: Las calificaciones bajas extremas pueden ser un predictor temprano de riesgo de abandono, mientras que la mayoría con buenas notas probablemente continuará hasta la graduación.

7. Curricular units 1st sem (without evaluations)
- Sesgo positivo extremo (≈ 8.72) y kurtosis muy alta (≈ 101.7), indicando que casi todos los estudiantes tienen 0 unidades sin evaluaciones, con unos pocos casos extremos.
- Insight: La mayoría está cumpliendo con las evaluaciones programadas, pero los casos con unidades sin evaluaciones podrían reflejar inasistencias o retrasos académicos que podrían afectar su progreso.

8. Curricular units 2nd sem (credited)
- Sesgo positivo elevado (≈ 4.49) y kurtosis alta (≈ 22.65).
- Insight: Similar al primer semestre, pocos estudiantes acumulan créditos avanzados temprano; podrían ser estudiantes sobresalientes o muy comprometidos. Los que no tienen créditos podrían estar en riesgo.
- Tiene una distribución extremadamente sesgada y curtosis altísima. La mayoría de estudiantes tiene 0, con unos pocos valores extremos. Esto aporta muy poca información general para el modelo, y puede actuar más como ruido que como predictor confiable. Podrías agruparlo, transformarlo o incluso descartarlo.

9. Curricular units 2nd sem (enrolled)
- Sesgo moderado positivo (≈ 0.81).
- Insight: La mayoría mantiene una carga académica estándar, pero los que se inscriben en muchas unidades podrían enfrentar riesgo de sobrecarga y estrés.

10. Curricular units 2nd sem (evaluations)
- Sesgo bajo (≈ 0.38), la distribución es más simétrica que en el primer semestre.
- Insight: La carga de evaluaciones es más uniforme; los estudiantes con más evaluaciones podrían necesitar más apoyo para evitar sobrecarga.

11. Curricular units 2nd sem (approved)
- Sesgo ligeramente positivo (≈ 0.27), indicando que la mayoría aprueba un número promedio de unidades.
- Insight: La aprobación del segundo semestre refleja continuidad en el rendimiento académico; las bajas aprobaciones podrían anticipar riesgo de abandono.

12. Curricular units 2nd sem (grade)
- Sesgo negativo moderado (≈ -1.17), similar al primer semestre, con mayoría de estudiantes obteniendo buenas notas.
- Insight: El rendimiento general sigue siendo alto; notas bajas persistentes podrían indicar riesgo académico.

13. Curricular units 2nd sem (without evaluations)
- Sesgo positivo alto (≈ 7.62), kurtosis alta (≈ 73.26).
- Insight: La mayoría completa todas las evaluaciones, pero unos pocos podrían estar retrasados o tener problemas de asistencia.

14. Unemployment rate
- Distribución cercana a simétrica (skew ≈ 0.19).
- Insight: Entorno macroeconómico estable, pero mayores tasas de desempleo podrían afectar indirectamente a estudiantes con trabajos parciales.

15. Inflation rate
- Sesgo ligero positivo (≈ 0.27).
- Insight: Variaciones leves en la inflación probablemente no afectan significativamente a los estudiantes, aunque podrían impactar en la capacidad de pago de matrícula.

16. GDP
- Sesgo negativo ligero (≈ -0.39).
- Insight: Cambios en el PIB podrían reflejar el entorno económico; caídas pronunciadas podrían afectar la financiación de estudios o becas.

### 1.3 ANALISIS RESPECTO A TARGET
#### 1.3.1 VARIABLES CATEGÓRICAS RESPECTO A TARGET

FUNCION PARA EL ANÁLISIS COMPARATIVO DE VARIABLES CATEGÓRICAS

In [ ]:
def plot_numericas_categoricas(df, categorical_col, target_col='Target', figsize=(10,6), cmap="Set2"):
    ct = pd.crosstab(df[categorical_col], df[target_col])
    ct_percent = ct.div(ct.sum(axis=1), axis=0) * 100

    ax = ct_percent.plot(kind="barh", stacked=True, figsize=figsize, cmap=cmap, edgecolor='white', width=0.7)

    plt.xlabel("Percentage (%)")
    plt.ylabel(categorical_col)
    plt.title(f"{categorical_col} vs {target_col}", fontsize=14, fontweight='bold')

    for i, row in enumerate(ct_percent.values):
        left = 0
        for j, val in enumerate(row):
            if val > 0:
                ax.text(left + val/2, i, f"{val:.1f}%", va='center', ha='center', color='white', fontsize=9, fontweight='bold')
                left += val

    totals = ct.sum(axis=1)
    for i, total in enumerate(totals):
        ax.text(102, i, f"Total: {total}", va='center', ha='left', fontsize=10, fontweight='bold', color='black')

    plt.xlim(0, 110)
    plt.legend(title=target_col, bbox_to_anchor=(1.05, 1), fontsize=10)
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


In [ ]:
plot_numericas_categoricas(df_students, 'Marital status')

### Marital Status and Academic success
- Lo más destacado es que los solteros son mayoría absoluta: concentran tanto la mayor cantidad de dropouts (1184) como de graduados (2015).
- En cambio, los casados y divorciados tienen números menores, pero muestran una mayor proporción relativa de abandono.
- El resto de categorías (facto union, separated, widow) son poco representativas y no aportan un patrón claro.

In [ ]:
plot_numericas_categoricas(df_students, 'Course')

### Course and Academic success
- Lo más llamativo es Nursing, con 118 dropouts frente a 548 graduados → es el curso con mayor éxito relativo de permanencia.
- En contraste, Informatics Engineering presenta un caso crítico: 92 dropouts frente a solo 14 graduados, lo que refleja altísima deserción.
- También destacan patrones negativos en Equiniculture (78 vs 42) y Basic Education (85 vs 57), donde el abandono supera a los graduados.
- La mayoría de carreras (Marketing, Animation, Tourism, Veterinary Nursing, etc.) muestran proporciones más equilibradas, aunque algunas con ligera inclinación al dropout.
- Casos muy pequeños como Biofuel Production Technologies (8 vs 1) tienen poca relevancia estadística.

In [ ]:
plot_numericas_categoricas(df_students, 'Debtor')

### Debtor and Academic success
- La diferencia es muy clara: los estudiantes sin deudas tienen mejores resultados (2108 graduados vs 1109 dropouts).
- En cambio, los que sí tienen deudas muestran una situación inversa y preocupante (312 dropouts frente a solo 101 graduados).
- Esto podria tener un gran peso en el modelo debido tanto al desequilibrio tan evidente entre ambas variables y al valor contextual racional derivado de que las dificultades económicas afectan directamente la permanencia en la universidad

In [ ]:
plot_numericas_categoricas(df_students, 'Scholarship holder')

### Scholarship holder and Academic success
- Los estudiantes sin beca tienen más casos de abandono (1,287) que de graduación (1,374), lo que refleja un riesgo elevado de dropout.
En cambio, los becados muestran una diferencia muy marcada: 835 graduados frente a solo 134 dropouts, es decir, una proporción de éxito mucho mayor.
- La beca parece actuar como un factor protector: reduce significativamente la probabilidad de abandono y aumenta la probabilidad de graduación. Esto tiene mucho sentido en el contexto, ya que recibir apoyo económico suele aliviar cargas financieras y permite concentrarse en los estudios.
- Otra opción es que acceder a la beca impida que te salgas del curso sin re abonar lo invertido por la beca.

In [ ]:
plot_numericas_categoricas(df_students, 'Application order')

### Application order and Academic success
- La gran mayoría de los estudiantes selecciona su primera opción de aplicación (choice 1), con 1,408 graduados frente a 1,053 dropouts. Las opciones posteriores tienen menos estudiantes, y los dropouts tienden a ser proporción relativamente más alta en choices 4, 5 y 6.
- La prioridad de aplicación refleja motivación o claridad en el objetivo: los estudiantes que eligieron su primera opción tienen más probabilidades de graduarse, mientras que los que seleccionaron opciones posteriores muestran un ligero aumento relativo de abandono.
- Esto sugiere que Application order puede ser un predictor útil, aunque su peso no será tan grande como variables económicas o académicas directas.

In [ ]:
plot_numericas_categoricas(df_students, 'Tuition fees up to date')

### Tuition fees up to date and Academic success
- Los datos muestran un patrón muy claro: mantener las cuotas al día se asocia fuertemente con la graduación, mientras que los incumplimientos coinciden mayoritariamente con el abandono.
- Esta variable es un fuerte indicador de riesgo académico y puede ser clave en modelos predictivos, ya que distingue casi de manera directa a los estudiantes que completan el curso de los que abandonan.

In [ ]:
plot_numericas_categoricas(df_students, 'Daytime/evening attendance')

### Daytime/evening attendance and Academic Success
- Se observa que la gran mayoría de los estudiantes asiste en horario diurno, y dentro de este grupo, la proporción de graduados es significativamente mayor que la de dropouts. En cambio, los estudiantes de horario vespertino tienen un balance mucho más cercano entre graduados y dropouts.
- El horario de asistencia podría reflejar factores como disponibilidad de tiempo, obligaciones laborales u otras responsabilidades, haciendo que esta variable tenga cierta capacidad predictiva sobre el éxito académico.

#### 1.3.2 VARIABLES NUMÉRICAS RESPECTO A TARGET

FUNCIÓN PARA ANÁLISIS COMPARATIVO DE VARIABLES NUMÉRICAS

In [ ]:
def analisis_numerica_target(df, numeric_col, target_col='Target', figsize=(12,5), palette='Set2'):
    stats = df.groupby(target_col)[numeric_col].agg(['mean','median','std','min','max'])
    print(f"\n=== Estadísticas de '{numeric_col}' por '{target_col}' ===")
    print(stats)
    
    plt.figure(figsize=figsize)
    sns.histplot(data=df, x=numeric_col, hue=target_col, kde=True, palette=palette, element='step', stat='density', common_norm=False)
    plt.title(f"KDE/Histograma de {numeric_col} por {target_col}")
    plt.xlabel(numeric_col)
    plt.ylabel("Densidad")
    plt.show()
    
    if df[target_col].nunique() == 2:
        target_map = {k:i for i,k in enumerate(df[target_col].unique())}
        corr = df[numeric_col].corr(df[target_col].map(target_map))
        print(f"Correlación de {numeric_col} con {target_col} (binaria): {corr:.3f}")
    else:
        print(f"No se calcula correlación: '{target_col}' no es binaria")

In [ ]:
analisis_numerica_target(df_students, "Age at enrollment")

### Age at enrollment
- Los estudiantes que abandonan son, en promedio, mayores (≈26 años) que los graduados (≈22 años).
- La correlación negativa (-0.267) sugiere que a mayor edad al ingresar, mayor riesgo de abandono.

Valor: La edad puede ser un indicador útil para priorizar seguimiento o soporte a estudiantes mayores.

In [ ]:
analisis_numerica_target(df_students, "Curricular units 1st sem (credited)")

### Curricular units 1st sem (credited)
- Los estudiantes que graduaron acumularon en promedio 0,85 créditos en el primer semestre (mediana 0), mientras que los que abandonaron promediaron 0,61 créditos (mediana 0). La desviación estándar es relativamente alta en ambos grupos (≈2,1–2,7), reflejando que existen algunos estudiantes con muchos créditos, pero la mayoría tiene pocos o ninguno.
- El rango muestra que tanto graduados como dropouts incluyen casos extremos (0–20 créditos para graduados, 0–18 para abandonos), indicando alta dispersión.
- La correlación con el Target es muy baja (0,047), lo que sugiere que, aunque puede haber cierta relación, la cantidad de créditos acreditados en el primer semestre por sí sola no es un predictor fuerte del éxito académico.

In [ ]:
analisis_numerica_target(df_students, "Curricular units 1st sem (evaluations)")

### Curricular units 1st sem (evaluations)
- En promedio, los estudiantes graduados realizaron unas 8,27 evaluaciones en el primer semestre (mediana 8), mientras que los que abandonaron hicieron ligeramente menos, unas 7,75 (mediana 8).
- Aunque la diferencia es pequeña, la desviación estándar más alta en los abandonos (≈4,9 frente a ≈3,8 en graduados) sugiere que este grupo es más heterogéneo: algunos hicieron muchas pruebas, mientras que otros prácticamente no se presentaron.
- El rango máximo también refleja que los graduados llegaron a realizar hasta 45 evaluaciones frente a 31 de los abandonados.
- La correlación con el Target es muy baja (0,06), por lo que el número de evaluaciones realizadas en sí mismo no parece ser un factor decisivo de éxito o abandono, aunque puede servir como variable de contexto en combinación con otras (por ejemplo, aprobadas o calificaciones).

In [ ]:
analisis_numerica_target(df_students, "Curricular units 1st sem (approved)")

### Curricular units 1st sem (approved)
- Los estudiantes que graduaron aprobaron en promedio 6 asignaturas en el primer semestre (mediana 6), mientras que los que abandonaron aprobaron apenas ≈2,5 asignaturas (mediana 2).
- La desviación estándar es similar en ambos grupos (~2.6–2.85), lo que indica que hay cierta dispersión, pero la diferencia de medias es significativa.
- El rango muestra que incluso los que abandonaron tienen algunos aprobados (0–21), mientras que los graduados alcanzan hasta 26.
- La correlación con el Target es 0.555, lo que indica que esta variable es bastante predictiva: a mayor número de asignaturas aprobadas en el primer semestre, mayor probabilidad de graduarse.

In [ ]:
analisis_numerica_target(df_students, "Curricular units 1st sem (grade)")

### Curricular units 1st sem (grade)
- Los estudiantes que graduaron tienen una media de 12.64 puntos en sus calificaciones del primer semestre (mediana 13), mientras que los que abandonaron promedian 7.25 puntos (mediana 10.93).
- La desviación estándar es más alta en los que abandonaron (~6.03 vs 2.7), indicando mayor dispersión y algunos casos con calificaciones bajas pero también algunas más altas.
- El rango muestra que ambos grupos tienen valores extremos (0–18), pero la media y mediana claramente diferencian a graduados y dropout.
- La correlación con el Target es 0.52, indicando que las notas del primer semestre son también un buen predictor del éxito académico, aunque ligeramente menos fuerte que la cantidad de asignaturas aprobadas.

In [ ]:
analisis_numerica_target(df_students, "Curricular units 1st sem (enrolled)")

### Curricular units 1st sem (enrolled)
- Los graduados y los dropouts se matriculan en un número similar de unidades el primer semestre: promedio 6.67 vs 5.82, mediana 6 en ambos casos.
- La dispersión es algo mayor en los graduados (std 2.66 vs 2.33), y ambos grupos incluyen estudiantes con 0 unidades matriculadas, lo que puede reflejar casos atípicos.
- La correlación con el Target es muy baja (0.161), indicando que el simple número de unidades en las que se matricula un estudiante no es un buen predictor de graduación o abandono.

In [ ]:
analisis_numerica_target(df_students, "Curricular units 1st sem (without evaluations)")

### Curricular units 1st sem (without evaluations)
- Los estudiantes que abandonaron presentan en promedio 0,19 asignaturas sin evaluación en el primer semestre, más del doble que los graduados (0,09). La mediana en ambos grupos es 0, lo que indica que la mayoría de los estudiantes sí se presentan a todas sus evaluaciones, pero cuando existen asignaturas sin evaluar, estas son más frecuentes en quienes terminan abandonando.
- La desviación estándar ligeramente más alta en los dropouts (0,79 frente a 0,59) refleja mayor dispersión: algunos casos extremos muestran hasta 8 asignaturas sin evaluación en abandonos, frente a un máximo de 12 en graduados, aunque en proporción mucho menor.
- La correlación negativa con el Target (-0,075) confirma que acumular asignaturas sin evaluación es un indicador, aunque débil, de mayor riesgo de abandono. Este patrón tiene lógica: no presentarse a exámenes puede reflejar desmotivación, sobrecarga académica o problemas externos que impactan en la continuidad de los estudios.

In [ ]:
analisis_numerica_target(df_students, "Curricular units 2nd sem (credited)")

### Curricular units 2nd sem (credited)
- En el segundo semestre, los graduados registran un promedio ligeramente superior de asignaturas convalidadas (0,67) frente a los estudiantes que abandonan (0,45). Sin embargo, en ambos casos la mediana es 0, lo que indica que la mayoría no tiene créditos reconocidos.
- La dispersión es algo mayor en los graduados (desviación estándar 2,21 frente a 1,68), lo que sugiere que existe un pequeño grupo de estudiantes que sí acumula convalidaciones en este periodo.
- La correlación con el Target es positiva pero muy baja (0,052), lo que refleja que el número de créditos convalidados tiene un peso prácticamente marginal en la predicción del éxito académico.
- Aunque no parece ser un factor decisivo, podría interpretarse que los estudiantes con más convalidaciones suelen estar mejor posicionados académicamente o provienen de trayectorias previas más sólidas, lo que puede facilitar la continuidad.

In [ ]:
analisis_numerica_target(df_students, "Curricular units 2nd sem (approved)")

### Curricular units 2nd sem (approved)
- Los estudiantes que graduaron aprobaron en promedio 6.18 asignaturas del segundo semestre (mediana 6), mientras que los que abandonaron apenas aprobaron 1.94 asignaturas (mediana 0).
- La desviación estándar indica que los dropouts tienen más dispersión (~2.57), con algunos aprobando varias asignaturas, pero la mayoría con pocas o ninguna.
- El rango confirma que ambos grupos tienen extremos (0–16 para dropouts, 0–20 para graduados), pero la media y mediana muestran una clara diferencia.
- La correlación con el Target es 0.654, la más alta hasta ahora, lo que indica que aprobar asignaturas en el segundo semestre es un fuerte predictor de graduación.

In [ ]:
analisis_numerica_target(df_students, "Curricular units 2nd sem (enrolled)")

### Curricular units 2nd sem (enrolled)
- En el segundo semestre, los graduados se matriculan en más asignaturas en promedio (6,63) que los estudiantes que abandonan (5,78). La mediana coincide en 6 para ambos grupos, lo que indica que la diferencia se da en los extremos: los graduados tienden a asumir cargas más altas, llegando hasta 23 materias, mientras que los dropouts se quedan más bajos (máximo 18).
- La desviación estándar es similar (2,29 vs 2,11), lo que refleja cierta variabilidad pero consistente en ambos grupos.
- La correlación con el Target es moderada y positiva (0,183), lo que sugiere que matricularse en un mayor número de asignaturas se relaciona con una mayor probabilidad de graduarse.
- En términos prácticos, podría interpretarse como un indicador de compromiso académico: los estudiantes que mantienen un ritmo alto de matrícula probablemente confían en su capacidad de aprobar y permanecer en el programa, mientras que quienes reducen la carga académica pueden estar enfrentando dificultades que desembocan en el abandono.

In [ ]:
analisis_numerica_target(df_students, "Curricular units 2nd sem (grade)")

### Curricular units 2nd sem (grade)
- Los estudiantes que graduaron tienen un promedio de calificaciones de 12.70 en el segundo semestre (mediana 13), mientras que los dropouts apenas llegan a 5.90 (mediana 0).
- La desviación estándar indica que entre los dropouts hay mucha dispersión (6.12), con algunos teniendo calificaciones altas pero muchos con 0, mientras que los graduados muestran más consistencia (2.69).
- El rango confirma que los dropouts pueden tener calificaciones altas aisladas, pero la mayoría está concentrada en valores bajos.
- La correlación con el Target es 0.605, también bastante alta, indicando que las calificaciones del segundo semestre son un buen predictor de graduación.

In [ ]:
analisis_numerica_target(df_students, "Curricular units 2nd sem (evaluations)")

### Curricular units 2nd sem (evaluations)
- Los estudiantes graduados se presentan a un mayor número de evaluaciones en el segundo semestre (media 8,14; mediana 8) que los dropouts (media 7,17; mediana 7). Aunque la diferencia no es abismal, sí es consistente, y además la desviación estándar muestra que los abandonos tienen mayor dispersión (4,81 frente a 3,24), es decir, un comportamiento más irregular en su asistencia a evaluaciones.
- La correlación positiva (0,119) confirma que cuantas más evaluaciones realiza un estudiante, mayor es la probabilidad de graduarse.
En términos prácticos, esta variable refleja el grado de compromiso: presentarse a las evaluaciones implica continuidad y esfuerzo sostenido, mientras que la ausencia de las mismas suele anticipar riesgo de abandono.

In [ ]:
analisis_numerica_target(df_students, "Curricular units 2nd sem (without evaluations)")

### Curricular units 2nd sem (without evaluations)
- Los estudiantes que abandonan presentan un promedio más alto de asignaturas matriculadas pero nunca evaluadas (0,24) en comparación con los graduados (0,08). Aunque la mediana es 0 para ambos grupos, la dispersión es mayor en los dropouts (desviación estándar 0,99 frente a 0,52), lo que indica que es más común que los estudiantes en riesgo acumulen materias sin presentarse a examen o sin finalizar el proceso.
- La correlación con el Target es negativa (-0,103), lo que significa que a mayor número de asignaturas sin evaluación, mayor probabilidad de abandono.
- Este patrón tiene bastante lógica: no asistir a evaluaciones es un claro indicador de desconexión académica, desmotivación o incluso problemas personales/financieros. En la práctica, podría usarse como una señal temprana para detectar riesgo de dropout y activar mecanismos de acompañamiento.

In [ ]:
analisis_numerica_target(df_students, "Unemployment rate")

### Unemployment rate
- Las tasas de desempleo son prácticamente idénticas entre estudiantes que abandonan (11.62%) y graduados (11.64%), con medianas de 11.1% y rangos idénticos (7.6–16.2%).
- La desviación estándar es similar para ambos grupos, mostrando poca variabilidad en la muestra.
- La correlación con el Target es casi nula (0.004), lo que indica que esta variable no tiene ningún poder predictivo sobre si un estudiante se graduará o abandonará.
- Esto nos tiende a indicar que no será importante para el modelo

In [ ]:
analisis_numerica_target(df_students, "Inflation rate")

### Inflation rate
- La inflación media es ligeramente mayor para los estudiantes que abandonan (1.28%) frente a los graduados (1.20%), pero la diferencia es mínima.
- Los valores medianos muestran una diferencia mayor (1.4 vs 0.6), pero los rangos son idénticos (-0.8 a 3.7%) y la desviación estándar es similar para ambos grupos.
- La correlación con el Target es muy baja (-0.03), indicando que la inflación prácticamente no influye en el éxito académico de los estudiantes.

In [ ]:
analisis_numerica_target(df_students, "GDP")

### GPD
- Los graduados presentan un PIB medio ligeramente positivo (0.08) mientras que los estudiantes que abandonan tienen un PIB medio ligeramente negativo (-0.15), aunque ambos grupos muestran rangos idénticos (-4.06 a 3.51) y desviaciones estándar similares (~2.25).
- La correlación con el Target es muy baja (0.05), indicando que el PIB no tiene un efecto relevante sobre la probabilidad de graduarse o abandonar.

# 2. PROCESAMIENTO DE DATOS

Despues de realizar pruebas respecto a la transformación a numéricas de las variable categóricas. La que ha demostrado obtener mejores resultados en los modelos ha sido convertir todas en dummies. Debido a que no son muchas las variables del dataset, esta no introduce demasiado ruido.

In [ ]:
categorical_cols = df_students.select_dtypes(['category', 'object']).columns.drop('Target')
df_encoded = pd.get_dummies(df_students, columns=categorical_cols, drop_first=True)

Forma que obtuvo menos score:

In [ ]:
df_engagement = df_encoded.copy()

In [ ]:
df_engagement['Target_binary'] = df_engagement['Target'].map({'dropout': 0, 'graduate': 1})
df_engagement = df_engagement.drop(['Target'], axis=1)

correlations_with_EngagementLevel = df_engagement.corr()['Target_binary'].sort_values(ascending=False)

correlation_table = correlations_with_EngagementLevel.to_frame(name='Correlation with EngagementLevel').reset_index()

correlation_table.rename(columns={'index': 'Feature'}, inplace=True)

display(correlation_table.head(30))

## 2.1 INSIGHTS CLAVE SOBRE LA TARGET
#### Correlaciones fuertes
- Curricular units 2nd sem (approved) (0.65) y Curricular units 2nd sem (grade) (0.61), junto con Curricular units 1st sem (approved) (0.55) y Curricular units 1st sem (grade) (0.52):
- El rendimiento académico en los dos primeros semestres es el predictor más importante de la graduación.
Implicación: intervenir en estudiantes con bajas notas o pocas aprobadas desde 1st sem puede reducir el abandono.

#### Correlaciones moderadas
- Tuition fees up to date_yes (0.44): estar al corriente en pagos se asocia con más probabilidad de graduarse.
- Scholarship holder_yes (0.31): los becados muestran mayor retención.

#### Correlaciones débiles
- Course_Nursing (0.21): algunos cursos, como Nursing, tienen mejores tasas de graduación.
- Curricular units 2nd sem (enrolled) (0.18) y Curricular units 1st sem (enrolled) (0.16): mayor número de matrículas indica mayor compromiso.
- Variables personales como Displaced_yes (0.13) o Marital status_single (0.12) muestran un impacto pequeño.

#### Variables marginales
- Father's occupation_Unskilled/Manual, Application order_choice 2/3/4/6, Application mode_Others, GDP, etc. (<0.07): contribuyen muy poco al resultado.

#### Conclusión breve:
- El éxito en los primeros semestres y la situación financiera (pagos y becas) son los predictores más potentes de la graduación. Factores sociodemográficos y de acceso al curso apenas añaden valor.

In [ ]:
correlations = df_engagement.corr()['Target_binary'].abs().sort_values(ascending=False)

top30_features = correlations.drop('Target_binary').head(12).index

engagement_top30 = df_engagement[top30_features].join(df_engagement['Target_binary'])
engagement_correlation_top30 = engagement_top30.groupby('Target_binary').mean().T

plt.figure(figsize=(12, 10))
sns.heatmap(
    engagement_correlation_top30, 
    annot=True, 
    cmap='YlGnBu_r', 
    fmt='.2f', 
    linewidths=0.5, 
    cbar_kws={"shrink": 1}
)
plt.title('Top 12 Feature Interaction with Engagement Level')
plt.ylabel('Features')
plt.xlabel('Engagement Level')
plt.tight_layout()
plt.show()

# 3. ENTRENAMIENTO DEL MODELO Y EVALUACION

Dividimos el dataset en train y test

In [ ]:
X = df_encoded.drop(['Target'], axis=1)
y = df_encoded['Target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

Definimos los modelos

In [ ]:
modelos = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

Entrenamos y evaluamos 

In [ ]:
resultados = {}

for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    
    print(f"\n=== {nombre} ===")
    
    cm = confusion_matrix(y_test, y_pred)
    print("Matriz de confusión:")
    print(cm)
    
    print("\nReporte:")
    print(classification_report(y_test, y_pred))
    
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="YlGnBu",
                xticklabels=y.unique(), 
                yticklabels=y.unique())
    plt.title(f"Matriz de confusión - {nombre}")
    plt.xlabel("Predicción")
    plt.ylabel("True")
    plt.show()

En este punto se identificó que los modelos con mejor rendimiento global eran Logistic Regression y Random Forest, ambos alcanzando un 92% de accuracy. Antes de seleccionar el modelo definitivo para su productivización, se consideró oportuno realizar un proceso de simplificación del dataset. El objetivo es doble: por un lado, facilitar la interpretabilidad y usabilidad del modelo en un entorno real (reducir el número de variables que deben introducirse en la aplicación); por otro, evaluar si la eliminación de variables poco relevantes puede contribuir a mejorar o mantener el rendimiento de los modelos.

## 3.1 IMPORTANCIAS DE LAS VARIABLES

Vamos a utilizar la importancia en los modelos de las variables para tratar de reducir las dimensionalidades del modelo al minimo

In [ ]:
rf = modelos["Random Forest"]
rf.fit(X_train, y_train)

importances = rf.feature_importances_
features = X_train.columns
feat_imp_df = pd.DataFrame({'Feature': features, 'Importance': importances})
feat_imp_df = feat_imp_df.sort_values(by='Importance', ascending=False)

# Mostrar top 10
print("Top 10 features Random Forest:")
print(feat_imp_df.tail(80))

# Gráfico
plt.figure(figsize=(8,6))
sns.barplot(x="Importance", y="Feature", data=feat_imp_df.tail(20))
plt.title("Top 10 Features - Random Forest")
plt.show()

In [ ]:
lr = modelos["Logistic Regression"]

lr.fit(X_train, y_train)

coef = lr.coef_[0]  #

import numpy as np
abs_coef = np.abs(coef)

import pandas as pd
feat_coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': coef,
    'AbsCoefficient': abs_coef
})

feat_coef_df = feat_coef_df.sort_values(by='AbsCoefficient', ascending=False)

print("Top 10 variables más importantes - Logistic Regression:")
print(feat_coef_df.tail(80))

In [ ]:
# Ordenar de mayor a menor importancia
feat_imp_df_sorted = feat_imp_df.sort_values(by='Importance', ascending=False)

pd.set_option('display.max_rows', None)  # Mostrar todas las filas
feat_imp_df_sorted

#### 1. Justificación para la primera tanda de DROPS
- Las variables eliminadas en esta primera fase se descartaron principalmente por baja relevancia en la predicción del modelo.
- Las categorías de estado civil (Marital status) mostraban coeficientes cercanos a cero en Logistic Regression y mínima importancia en Random Forest, indicando que su contribución a la predicción de dropout o graduate es prácticamente nula.
- Las opciones de Application order_choice y Previous qualification con baja representación o poca relevancia no aportaban información significativa al modelo.
- Algunos cursos y asistencia diurna/nocturna presentaban importancia muy residual según Random Forest, por lo que eliminarlas simplifica el dataset sin afectar el rendimiento.
En conjunto, esta limpieza inicial permitió reducir el número de variables irrelevantes, facilitando la interpretabilidad y agilizando la fase de modelización.

In [ ]:
# Lista de columnas a eliminar
cols_to_drop = [
    # Marital status
    'Marital status_facto union', 'Marital status_legally separated',
    'Marital status_married', 'Marital status_single', 'Marital status_widow',
    # Application order_choice bajas
    'Application order_choice 5', 'Application order_choice 9',
    # Previous qualification bajas
    'Previous qualification_Secondary', 'Previous qualification_Higher',
    'Previous qualification_Other/Spec',
    # Cursos poco relevantes
    'Course_Biofuel Production Technologies', 'Course_Oral Hygiene',
    'Course_Agronomy', 'Course_Social Service (evening attendance)',
    # Daytime/evening attendance
    'Daytime/evening attendance_evening'
]

df_encoded_2 = df_encoded.copy()

df_encoded_2.drop(columns=cols_to_drop, inplace=True)

In [ ]:
X = df_encoded_2.drop(['Target'], axis=1)
y = df_encoded_2['Target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

modelos = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
}

resultados = {}

for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    
    print(f"\n=== {nombre} ===")
    
    cm = confusion_matrix(y_test, y_pred)
    print("Matriz de confusión:")
    print(cm)
    
    print("\nReporte:")
    print(classification_report(y_test, y_pred))
    
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="YlGnBu",
                xticklabels=y.unique(), 
                yticklabels=y.unique())
    plt.title(f"Matriz de confusión - {nombre}")
    plt.xlabel("Predicción")
    plt.ylabel("True")
    plt.show()

#### 2. Justificación para la segunda y definitiva tanda (cols_to_drop_more)
- En esta segunda fase, se aplicó un filtrado más exhaustivo y sistemático para eliminar familias completas de variables categóricas que mostraban importancia residual muy baja (<0.01) en Random Forest y coeficientes casi nulos en Logistic Regression.
- Se eliminaron todos los cursos con baja relevancia, todas las opciones de Application order_choice y Application mode poco significativas, así como todas las categorías de Marital status y algunas ocupaciones/estudios de padres/madres irrelevantes.
- También se descartaron otras variables marginales como Displaced_yes o Daytime/evening attendance_evening.
Esta reducción asegura un dataset más compacto y manejable, mantiene la performance del modelo prácticamente intacta y mejora la interpretabilidad, lo que facilita la explicación de resultados a negocio y la productivización en una ap

In [ ]:
cols_to_drop_more = [
    # Cursos con importancia residual
    'Course_Animation and Multimedia Design', 'Course_Basic Education',
    'Course_Equiniculture', 'Course_Informatics Engineering', 'Course_Journalism and Communication',
    'Course_Management', 'Course_Management (evening attendance)',
    'Course_Nursing', 'Course_Oral Hygiene', 'Course_Social Service',
    'Course_Tourism', 'Course_Veterinary Nursing', 'Course_Communication Design',
    'Course_Agronomy', 'Course_Biofuel Production Technologies', 'Course_Social Service (evening attendance)',
    
    # Application order_choice bajas
    'Application order_choice 1', 'Application order_choice 2', 'Application order_choice 3',
    'Application order_choice 4', 'Application order_choice 5', 'Application order_choice 6',
    'Application order_choice 9',
    
    # Application mode bajas
    'Application mode_Others', 'Application mode_2nd phase - general contingent',
    'Application mode_3rd phase - general contingent', 'Application mode_Change of course',
    'Application mode_Change of institution/course', 'Application mode_Holders of other higher courses',
    'Application mode_Over 23 years old', 'Application mode_Technological specialization diploma holders',
    'Application mode_Transfer',
    
    # Familia Marital status (todas)
    'Marital status_facto union', 'Marital status_legally separated',
    'Marital status_married', 'Marital status_single', 'Marital status_widow',
    
    # Familias de padres/madres poco relevantes
    'Mother\'s occupation_Other/Unknown', 'Mother\'s occupation_Specialized/Prof', 'Mother\'s occupation_Unskilled/Manual',
    'Father\'s occupation_Armed Forces', 'Father\'s occupation_Other/Unknown', 'Father\'s occupation_Specialized/Prof',
    'Father\'s occupation_Unskilled/Manual',
    'Mother\'s qualification_Other/Unknown', 'Mother\'s qualification_Secondary',
    'Father\'s qualification_Higher', 'Father\'s qualification_Secondary', 'Father\'s qualification_Unknown/Other',
    
    # Otros
    'Displaced_yes', 'Daytime/evening attendance_evening',
    # Variables macroeconómicas (impacto marginal)
    'Unemployment rate', 'Inflation rate', 'GDP'
]

df_encoded_3 = df_encoded_2.drop(columns=cols_to_drop_more, errors='ignore')

## 3.2 MODELO DEFINITIVO

In [ ]:
X = df_encoded_3.drop(['Target'], axis=1)
y = df_encoded_3['Target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

modelos = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
}

resultados = {}

for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    
    print(f"\n=== {nombre} ===")
    
    cm = confusion_matrix(y_test, y_pred)
    print("Matriz de confusión:")
    print(cm)
    
    print("\nReporte:")
    print(classification_report(y_test, y_pred))
    
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="YlGnBu",
                xticklabels=y.unique(), 
                yticklabels=y.unique())
    plt.title(f"Matriz de confusión - {nombre}")
    plt.xlabel("Predicción")
    plt.ylabel("True")
    plt.show()

## 3.3 MODELOS DESCARTADOS
En la primera fase de experimentación, antes de la depuración de variables, se evaluaron cuatro algoritmos: Random Forest, Logistic Regression, Gradient Boosting y KNN. Tras una primera revisión de resultados, se decidió descartar Gradient Boosting y KNN por las siguientes razones:

- Gradient Boosting:
Aunque obtuvo un rendimiento aceptable (accuracy ≈ 90%), mostró un recall más bajo para la clase dropout (0.82), lo cual es crítico dado que el objetivo principal es identificar estudiantes en riesgo de abandono. Además, el modelo es más complejo de interpretar y ajustar respecto a Random Forest y Logistic Regression, lo que no justificaba su inclusión como candidato final.

- KNN (K-Nearest Neighbors):
Presentó el rendimiento más bajo entre los cuatro modelos (accuracy ≈ 87%) y un recall especialmente reducido en la clase dropout (0.74), lo que significa que dejó de identificar una proporción significativa de abandonos reales. Además, KNN tiende a escalar mal con grandes volúmenes de datos y carece de interpretabilidad en comparación con los modelos restantes.

Tras la depuración final de variables, se repitió la comparación entre los modelos candidatos (Random Forest y Logistic Regression). Ambos mantuvieron un rendimiento muy similar en términos globales (accuracy ≈ 92%), pero se decidió descartar Random Forest y continuar únicamente con Logistic Regression por las siguientes razones:

- Precisión global: Logistic Regression mantuvo una accuracy del 92%, muy similar a Random Forest, pero con métricas de equilibrio ligeramente más consistentes entre ambas clases.
- Balance entre clases: Logistic Regression alcanzó un recall superior en la clase dropout (0.85 vs 0.83). Dado que el objetivo principal del trabajo es identificar estudiantes en riesgo de abandono, esta métrica tiene un peso clave en la decisión.
- Interpretabilidad: Logistic Regression permite interpretar directamente los coeficientes de las variables y cuantificar su impacto en la probabilidad de abandono o graduación. Random Forest, aunque robusto, requiere técnicas adicionales (importancias de variables, SHAP values) que dificultan la comunicación de los resultados a responsables académicos o institucionales.

In [ ]:
lr = modelos["Logistic Regression"]
lr.fit(X_train, y_train)

coef = lr.coef_[0]
abs_coef = np.abs(coef)

feat_coef_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Coefficient": coef,
    "AbsCoefficient": abs_coef,
    "OddsRatio": np.exp(coef)  
})

feat_coef_df = feat_coef_df.sort_values(by="AbsCoefficient", ascending=False)

pd.set_option("display.max_rows", None)   # opcional, para ver todas
pd.set_option("display.float_format", lambda x: f"{x:.4f}")  # formato bonito

feat_coef_df.head(30)

# 4. Principales insights

1. Estado financiero → factor crítico
- Tuition fees up to date_yes (coef 2.296, OR ≈ 9.93):
  Los estudiantes con tasas pagadas al día tienen 10 veces más probabilidad de graduarse.
- Debtor_yes (coef -1.023, OR ≈ 0.36):
Tener deudas reduce la probabilidad de graduación a un 36% respecto a los no deudores.
👉 Insight: la gestión de pagos y la deuda son los predictores más fuertes del abandono.

2. Rendimiento académico temprano importa muchísimo

- Curricular units 2nd sem (approved) (coef 1.062, OR ≈ 2.89):
Aprobar asignaturas en el 2º semestre casi triplica la probabilidad de graduación.
- Curricular units 1st sem (approved) (coef 0.628, OR ≈ 1.87):
Aprobar en el 1º semestre casi duplica la probabilidad.
- En cambio, Curricular units 2nd sem (enrolled) (coef -0.892, OR ≈ 0.41):
Estar matriculado en muchas asignaturas sin aprobarlas reduce fuertemente la probabilidad de graduarse.
👉 Insight: lo que el estudiante haga en los dos primeros semestres (aprobar vs. solo matricularse) es determinante para su éxito.

3. Apoyo económico externo como factor protector
- Scholarship holder_yes (coef 0.727, OR ≈ 2.07):
Ser becario duplica las probabilidades de graduación.
👉 Insight: las becas no solo apoyan económicamente, sino que se asocian a una mayor retención.

4. Variables sociodemográficas
- Gender_male (coef -0.355, OR ≈ 0.70):
Ser hombre reduce en un 30% la probabilidad de graduación frente a mujeres.
- Age at enrollment (coef -0.015, OR ≈ 0.98 por año):
Los estudiantes de mayor edad tienen una ligera pero consistente menor probabilidad de graduación.

👉 Insight: perfiles de mayor riesgo → hombres y mayores de edad en el ingreso.

5. Notas y evaluaciones tienen menor peso
- 1st sem (grade) (coef -0.114, OR ≈ 0.89) y 2nd sem (grade) (coef 0.081, OR ≈ 1.08):
Las calificaciones tienen efecto, pero mucho más débil que los aprobados/inscritos.
- Evaluations (coef cercanos a 0): apenas influyen.

👉 Insight: no basta con presentarse a exámenes → lo que realmente marca la diferencia es aprobar y avanzar.

# 5 PRODUCTIVIZACIÓN MODELO

In [ ]:
logistic_model = LogisticRegression(max_iter=1000, random_state=42)

logistic_model.fit(X_train, y_train)

joblib.dump(logistic_model, "modelo_logistic.pkl")

In [ ]:
df_students.to_csv("df_students.csv", index=False)

# RESTO DE INTENTOS CADUCOS